### 1. Importing Libraries

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

from geopy.geocoders import Nominatim 
import pgeocode

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

import requests
import bs4 as bs

### 2. Retrieve rental data using Selenium and BeautifulSoup 

References here: - https://bit.ly/2ZFM0kY - https://realpython.com/beautiful-soup-web-scraper-python/

In [14]:
df = pd.read_excel('uni_toronto_final.xlsx')
rental = {}

# for purpose of testing, we are going to grab just the first page of the listing.
for lat, long, uni in zip(df['Latitude'], df['Longitude'], df['University']):

    url = f'https://www.zumper.com/apartments-for-rent/toronto-on?property-categories=apartment,house,room&sort=price&box={long},{lat},-79.2492457525948,43.79486799352122'

    options = webdriver.ChromeOptions()
    options = webdriver.ChromeOptions() 
    options.add_argument("start-maximized")
    options.add_argument('disable-infobars')
    driver=webdriver.Chrome(options=options, executable_path=r'/Users/apple/Downloads/chromedriver')
    driver.get(url)

    print(f'Retrieving Rental for: {uni}...')

    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[1]/div/div/div[2]/div[1]/div/div[1]/div/div/div[1]/div[5]/button")))

    page_source = driver.page_source
    
    soup = bs.BeautifulSoup(page_source, 'html.parser')
    result = soup.findAll('div', class_ = 'ListItemDesktopView_content__2zZTH')

    for i in result:
        # Each job_elem is a new BeautifulSoup object.
        # You can use the same methods on it as you did before.

        price = i.find('div', class_='ListItemDesktopView_price__3TSLy')
        apartment_type = i.find('div', class_='ListItemDesktopView_bedBathText__2pdpd')
        #address = i.find('div', class_='ListItemDesktopView_address__QRRqh')
        header = i.find('a', class_='ListItemDesktopView_headerText__3W3Oe')
        description = i.find('div', class_='ListItemDesktopView_overlayInfo__1LVha')

        # Add University
        if 'University' in rental.keys():
            rental['University'].append(uni)
        else:
            rental['University'] = []
            rental['University'].append(uni)
        
        # Add address
        if 'Address' in rental.keys():
            rental['Address'].append(header.text)
        else:
            rental['Address'] = []
            rental['Address'].append(header.text)
            
        # Add Room type
        if 'Type' in rental.keys():
            rental['Type'].append(apartment_type.text)
        else:
            rental['Type'] = []
            rental['Type'].append(apartment_type.text)
        
        # Add Price
        if 'Price' in rental.keys():
            rental['Price'].append(price.text)
        else:
            rental['Price'] = []
            rental['Price'].append(price.text)
            
        # Add Description
        if 'Description' in rental.keys():
            rental['Description'].append(description.text)
        else:
            rental['Description'] = []
            rental['Description'].append(description.text)

    # close selenium window
    print('Done..')
    print()
    driver.close()

Getting data for: University of Toronto.
Getting data for: York University.
Getting data for: Ryerson University.
Getting data for: Humber College.
Getting data for: Centennial College.
Getting data for: George Brown College.
Getting data for: OCAD University.
Getting data for: Michener Institute for Applied Health Sciences.
Getting data for: Canadian Memorial Chiropractic College.
Getting data for: Tyndale College & Seminary.
Getting data for: Regis College University of Toronto.
Getting data for: Seneca College.
Getting data for: Victoria University.
Getting data for: Academy of Design Toronto.
Getting data for: University of Trinity College.
Getting data for: Sprott Shaw Language College.
Getting data for: Upper Career College Business & Technology.
Getting data for: International Language Academy of Canada.
Getting data for: Upper Madison College.
Getting data for: Institute for Christian Studies.
Getting data for: University of St. Michael's College.
Getting data for: Wycliffe Col

In [56]:
# Converting rental dictionary to a python dataframe and saving it to excel.
temp = pd.DataFrame(rental)
temp.to_excel('raw_rental_data.xlsx', index = None)

### 3. Getting the coordinates from address using Geopy and pgeocode

More here: https://pypi.org/project/geopy/, https://pypi.org/project/pgeocode/

We will not able to get an accurate coordinate for some of the rental based on the address given but lucky for us, pgeocode will be able to extract the longitude and latitude coordinates from the Postal Code.

In [12]:
rental_data = pd.read_excel('raw_rental_data.xlsx')
rental_data['latitude'] = 0
rental_data['longitude'] = 0

for index, rows in rental_data.iterrows():
     
    postal = rows['Address'].split(',')[-1] # get the last value
    rental_data.loc[index, 'Postal Code'] = postal

    try: # if the value is a postal code
        nomi = pgeocode.Nominatim('CA') # specifying the country code for Canada
        location = nomi.query_postal_code(postal) 

        lat = location.at['latitude']
        long = location.at['longitude']
        
        rental_data.loc[index,'latitude'] = lat
        rental_data.loc[index,'longitude'] = long
        
    except:
        print('Coordinates not found...')

Let's have a preview the data that we have

In [13]:
rental_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456 entries, 0 to 455
Data columns (total 8 columns):
University     456 non-null object
Address        456 non-null object
Type           456 non-null object
Price          456 non-null object
Description    456 non-null object
latitude       380 non-null float64
longitude      380 non-null float64
Postal Code    456 non-null object
dtypes: float64(2), object(6)
memory usage: 28.6+ KB


A preview of the data shows that a few of the rentals do not have the coordinates due to the address. In such cases, I will be dropping the rows for now but for future improvements, I will take care of such data.

In [14]:
clean_rental_data = rental_data.dropna(subset=['latitude', 'longitude'])
clean_rental_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 380 entries, 0 to 455
Data columns (total 8 columns):
University     380 non-null object
Address        380 non-null object
Type           380 non-null object
Price          380 non-null object
Description    380 non-null object
latitude       380 non-null float64
longitude      380 non-null float64
Postal Code    380 non-null object
dtypes: float64(2), object(6)
memory usage: 26.7+ KB


Now that we have the data cleaned and prepared for the next step, let's export the dataframe to excel.

In [43]:
# Reorganising the columns. Dropping the "Description" column as it provides no further help in our analysis at this point
clean_rental_data = clean_rental_data[['University', 'Address', 'Postal Code', 'latitude', 'longitude','Type', 'Price']]

clean_rental_data.to_excel('clean_rental_data.xlsx', index = None)

Upon initial inspection of the data, some of the coordinates seem to be duplicated for locations that have a similar starting postal code. To avoid multiple points within a single location–which will be a problem for us to take a look at the pop up information in the map–we will be dropping the first instance of the duplicate location

In [44]:
clean_rental_data.drop_duplicates(subset = ['latitude', 'longitude' ], keep = 'last' ).reset_index(drop = True).count()

University     35
Address        35
Postal Code    35
latitude       35
longitude      35
Type           35
Price          35
dtype: int64

Wow! It would seem that there are a multiple duplicates in rental houses for the universities. Afraid that we may have dropped some of the universities from the dataset, let's include University as one of the subset.

In [45]:
clean_rental_data.drop_duplicates(subset = ['University', 'latitude', 'longitude'], keep = 'last' ).reset_index(drop = True).count()

University     253
Address        253
Postal Code    253
latitude       253
longitude      253
Type           253
Price          253
dtype: int64

Much better! The total number of records is much lesser than what we have initially but it is a good start. Let's save the data and proceed to the next step which is to calculate the distance between the rental to the university.

In [1]:
to_export = clean_rental_data.drop_duplicates(subset = ['University', 'latitude', 'longitude'], keep = 'last' ).reset_index(drop = True)

# adding Rental Names
count = 1
for index, rows in to_export.iterrows():
    
    to_export.loc[index, 'Rental Name'] = 'Rental #' + str(count)
    count += 1

to_export.to_excel('final_rental_data.xlsx', index = None)

NameError: name 'clean_rental_data' is not defined